In [ ]:
from fer import FER
import matplotlib.pyplot as plt
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def analyze_emotions(test_image):
    test_image = cv2.rotate(test_image, cv2.ROTATE_90_CLOCKWISE)

    emo_detector = FER(mtcnn=True)
    captured_emotions = emo_detector.detect_emotions(test_image)

    if captured_emotions:
        return captured_emotions[0]['emotions']
    return None

def process_batch(batch_frames, batch_times):
    batch_emotions = {}
    batch_results = []

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(analyze_emotions, batch_frames))

    for frame, emotions, time in zip(batch_frames, results, batch_times):
        if emotions:
            for emotion, value in emotions.items():
                batch_emotions[emotion] = batch_emotions.get(emotion, 0) + value
            batch_results.append((frame, emotions, time))

    if batch_emotions:
        dominant_emotion = max(batch_emotions, key=batch_emotions.get)
        dominant_frame = max(batch_results, key=lambda x: x[1].get(dominant_emotion, 0))

        print(f"Batch Summary:")
        print(f"  Dominant Emotion - {dominant_emotion}")
        print(f"  Emotion Details - {batch_emotions}")
        print(f"  Dominant Frame Time - {dominant_frame[2]:.2f} seconds")
        print(f"  Dominant Frame Emotion Details - {dominant_frame[1]}")

        plt.figure()
        plt.imshow(cv2.cvtColor(dominant_frame[0], cv2.COLOR_BGR2RGB))
        plt.axis('on')
        plt.show()

def process_video(video_path, frames_per_second=5, batch_size=10):
    # Открываем видео
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    batch_frames = []
    batch_times = []

    count = 0
    while count < duration * frames_per_second:
        video.set(cv2.CAP_PROP_POS_MSEC, count * 1000 / frames_per_second)
        success, frame = video.read()
        if not success:
            break

        batch_frames.append(frame)
        batch_times.append(count / frames_per_second)
        count += 1

        # Обработка батчей
        if len(batch_frames) == batch_size:
            process_batch(batch_frames, batch_times)
            batch_frames = []
            batch_times = []

    video.release()

# Пример использования
video_path = 'IMG_5262.MOV'
process_video(video_path, frames_per_second=10, batch_size=10)
